In [2]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/lib/python3.10/getpass.py", line 77, in unix_getpass
    passwd = _raw_input(prompt, stream, input=input)
  File "/usr/lib/python3.10/getpass.py", line 146, in _raw_input
    line = input.readline()
KeyboardInterrupt

During han

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [4]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [5]:
from unsloth import FastLanguageModel
import torch
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from dataclasses import dataclass, field
from typing import Optional
import torch
from peft import LoraConfig
from tqdm import tqdm
import pandas as pd
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import re
from transformers import TextStreamer
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [5]:
model = AutoModel.from_pretrained(
    "cychiuak/lora_model" # tag name, or branch name, or commit hash
)

NameError: name 'AutoModel' is not defined

In [ ]:
alpaca_prompt = """Below is a classification task. Answer only positive, neutral or negative, do not say anything else .

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [9]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "cychiuak/llama3B4FineTuning", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "### Classify the sentiment of the following sentence as negative, neutral or positive.", # instruction
        "Although this stock recorded a history high revenue, I don't think it will last long.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.45.0.dev0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<|begin_of_text|>Below is a classification task. Answer only positive, neutral or negative, do not say anything else.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
The Eiffel Tower is a famous tall tower in Paris. It is one of the most recognizable landmarks in the world and is known for its stunning views of the city.<|end_of_text|>


In [ ]:
validation_data = pd.read_csv('validation.csv', sep='\t')
print(validation_data)
answers = []
for i in range(len(validation_data['text'])):
# for i in range(5):
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          "### Classify the sentiment of the following sentence as negative, neutral or positive.", # instruction
          validation_data['text'][i], # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  textOutput = tokenizer.batch_decode(outputs)
  print("textOutput is",textOutput)
  match = re.search(r"### Response:\n(.*?)<\|end_of_text\|>", textOutput[0], re.DOTALL)

  if match:
      response_text = match.group(1).strip()
      print("response_text is ",response_text)
      response_text = response_text.lower()
      if("positive" in response_text):
        print("positive")
        answers.append("positive")
      elif("neutral" in response_text):
        print("neutral")
        answers.append("neutral")
      elif("negative" in response_text):
        print("negative")
        answers.append("negative")
      else:
        answers.append("error")
        print("Response section not found.")

correctPre = 0
for i in range(len(answers)):
  if(answers[i] == validation_data['label'][i]):
    correctPre += 1
print(correctPre)
accuracy = correctPre / len(answers)
print("accuracy is",accuracy)

                                                  text     label
0    According to Scanfil , demand for telecommunic...  negative
1    Kemira 's partner in the project is St. Peters...   neutral
2    Finnish Rautaruukki has been awarded a contrac...  positive
3    It is also 7.7 pct above the 12-month volume w...  positive
4    Operating profit for the three-month period de...  negative
..                                                 ...       ...
373  Benefon , a wireless and GPS technology compan...  positive
374  In 2008 , Kemira recorded revenue of approxima...   neutral
375  EB announced in its stock exchange release on ...  negative
376  The group will expand the chain to include 45 ...  positive
377  The buyer is real estate owner Propertos Oy , ...   neutral

[378 rows x 2 columns]
textOutput is ['<|begin_of_text|>Below is a classification task. Answer only positive, neutral or negative, do not say anything else.\n\n### Instruction:\n### Classify the sentiment of the followi

In [ ]:
userInput = "The stock price is about to go down!"
inputs = tokenizer(
[
    alpaca_prompt.format(
        "### Classify the sentiment of the following sentence as negative, neutral or positive.", # instruction
        userInput, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
textOutput = tokenizer.batch_decode(outputs)
output = re.search(r"### Response:\n(.*?)<\|end_of_text\|>", textOutput[0], re.DOTALL).group(1).strip()
print("input is ",userInput)
print("output is ",output)

In [ ]:
from sklearn.metrics import confusion_matrix
label = validation_data['label']
print(confusion_matrix(label,answers,labels=["positive", "neutral", "negative"]))

In [ ]:
import numpy as np

def calculate_f1_scores(confusion_matrix):
    num_classes = confusion_matrix.shape[0]
    f1_scores = []
    
    for i in range(num_classes):
        true_positive = confusion_matrix[i, i]
        false_positive = sum(confusion_matrix[:, i]) - true_positive
        false_negative = sum(confusion_matrix[i, :]) - true_positive
        
        precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
        recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)
    
    macro_f1 = np.mean(f1_scores)
    return f1_scores, macro_f1

# Example usage:
confusion_matrix = np.array([
    [75, 33, 0],
    [9, 203, 3],
    [0, 6, 49]
])

f1_scores, macro_f1 = calculate_f1_scores(confusion_matrix)
print("F1 Scores for each class:", f1_scores)
print("Macro F1 Score:", macro_f1)